In [ ]:
import glob
import pandas

In [ ]:
glob.glob("*.csv")

In [ ]:
events = pandas.read_csv(
    'gfd_event_stats_20215_13_error_fixed_2.csv', 
    parse_dates=['dfo_began', 'dfo_ended'], 
    dayfirst=False,
    usecols=["system:index",'dfo_began', 'dfo_ended', 'dfo_cause']
).set_index("system:index")
events['duration_days'] = (events.dfo_ended - events.dfo_began).astype('timedelta64[D]').astype('int')
events['year'] = pandas.DatetimeIndex(events.dfo_began).year

In [ ]:
country_events = pandas.read_csv('country_events.csv').set_index('DFO').join(events)
country_events.head()

In [ ]:
annual_disruption = country_events.groupby(["ISO_A3", "dfo_cause", "year"]).sum(numeric_only=True).reset_index()

In [ ]:
full_combo = []
from itertools import product
for iso, cause, year in product(
    annual_disruption.ISO_A3.unique(), annual_disruption.dfo_cause.unique(), range(2000, 2019)):
    full_combo.append({
        'ISO_A3': iso,
        'dfo_cause': cause,
        'year': year
    })
full_combo = pandas.DataFrame(full_combo)
full_combo

In [ ]:
all_years_durations = full_combo.merge(annual_disruption, on=["ISO_A3","dfo_cause","year"], how='left').fillna(0)

In [ ]:
def p10(x):
    return x.quantile(0.1)

def p90(x):
    return x.quantile(0.9)

In [ ]:
event_summary = (
    country_events
    .groupby(["ISO_A3", "dfo_cause"])
    ['duration_days']
    .agg(['min', p10, 'median', 'mean', p90, 'max', 'count'])
)
event_summary.to_csv("country_event_durations.csv")

annual_summary = (
    all_years_durations
    .groupby(["ISO_A3", "dfo_cause"])
    ['duration_days']
    .agg(['min', p10, 'median', 'mean', p90, 'max', 'count'])
)
annual_summary.to_csv("country_annual_durations.csv")